<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week14.do.txt --no_mako -->
<!-- dom:TITLE: May 1-5: Advanced machine learning and data analysis for the physical sciences -->

# May 1-5: Advanced machine learning and data analysis for the physical sciences
**Morten Hjorth-Jensen**, Department of Physics and Center for Computing in Science Education, University of Oslo, Norway and Department of Physics and Astronomy and Facility for Rare Isotope Beams, Michigan State University, East Lansing, Michigan, USA

Date: **May 1-5, 2023**

## Plans for the week May 1-5

1. Gaussian processes and Bayesian statistics

2. [Video of lecture](https://youtu.be/)

3. [Whiteboard notes](https://github.com/CompPhysics/AdvancedMachineLearning/blob/main/doc/HandwrittenNotes/NotesApr262023.pdf)